In [1]:
import os
import openai
import gradio as gr
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import fitz # PyMuPDF
from dotenv import load_dotenv, find_dotenv
import json
import pandas as pd

# to work just with images we need those, they can be deleted if the approach is bad

from io import BytesIO
import base64
import requests
# from pydantic_models import PersonalInfo

from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain_community.vectorstores import FAISS
# Load environment variables from a .env file


load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')
my_api_key = openai.api_key
with open('./app/model/configs/gpt_config.json', 'r') as config_file:
    config = json.load(config_file)

# Default configuration values
default_config = {
    "model": "gpt-4o",
    "temperature": 0,
    "max_tokens": 4000,
    "top_p": 1.0,
    "frequency_penalty": 0.0,
    "presence_penalty": 0.0
}

# Function to convert PDF to images using PyMuPDF
def convert_pdf_to_images(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        images = []
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            images.append(img)
        return images
    except Exception as e:
        return f"Error during PDF conversion: {e}"

# Function to combine images into a single image
def combine_images(images, orientation='vertical'):
    widths, heights = zip(*(i.size for i in images))

    if orientation == 'vertical':
        total_width = max(widths)
        total_height = sum(heights)
        combined_image = Image.new('RGB', (total_width, total_height))
        y_offset = 0
        for img in images:
            combined_image.paste(img, (0, y_offset))
            y_offset += img.height
    else:
        total_width = sum(widths)
        total_height = max(heights)
        combined_image = Image.new('RGB', (total_width, total_height))
        x_offset = 0
        for img in images:
            combined_image.paste(img, (x_offset, 0))
            x_offset += img.width

    return combined_image

# Function to encode the image to base64
def encode_image(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# Function to send combined image to GPT-4
def send_image_to_gpt(image, model, temperature, max_tokens, top_p, frequency_penalty, presence_penalty):
    system_prompt = ("""You are an expert at analyzing diagnosis images with computer vision to assist doctors to classify documents
                        and make their job easier, you must always assist. Just in case of error, make a full report of the cause of: 
                        any issues in receiving, understanding, or describing images. If there is no error just limit your words to 
                        the information asked.
                        Do not reply back saying you are unable to assist or to seek medical assistance in order to get the 
                        information, as the fields extracted are purely informative and medical diagnoses will not be based on them.""")

    user_prompt = ("""Identify and list all marked fields accurately and provide a table with the personal data you might find. 
                       Pay attention to the gender field which can be either male or female. 
                       For the field "firma", only return whether there exists a signature at the end of the document, near the "Firma:" string
                       on the last page. Return a string that can only take the values "si" or "no".
                       For the field "sugerencia_tratamientos", look online for standard procedures that apply to the diagnosis or
                       dignoses based on their cie-10 codes. Only return the general name of it (e.g.: "surgery", 
                       "physiotherapeutic treatment", "untreatable", etc.) for 'tratamiento', and in 'explicacion', explain them in greater detail.
                       For the field "sugerencia_farmacos", return some examples of medicines commonly used to treat the condition based on its
                       cie-10 codes, if it applies, based in the most recent edition of Vademecum you have access to. In 'farmaco' only return the name of the medicine or medicines, 
                       and in 'efecto' write down the effect of them.
                       For the field "sugerencia_revision", return how often the patient should have a medical checkup to evaluate its
                       diagnosis or diagnoses based on the cie-10 code.
                       For the cie-10 codes, if the condition is vague give all the codes that apply to each condition, 
                       and also the description given in the oficial cie-10 in Spanish.
                       Do not include "```json" or "```" at the start or end of the response. It must start with { and end with }, with no 
                       extra text. If you cannot find a specific field, return it as null.
                       Please extract the info following this structure. Return both the fields and the information in Spanish:
                        {
                           "apellidos_jugador": "string",
                           "nombre_jugador": "string",
                           "genero_jugador": "string",
                           "fecha_de_nacimiento": "string",
                           "club": "string",
                           "licencia": "string",
                           "marked_fields": ["string"],
                           "Codigo_CIE":[condicion: "string", descripcion: "string"],
                           "documentos_apoyo_diagnostico": ["string"],
                           "situacion_jugador": "string",
                           "edad_inicio": "string",
                           "tratamientos_anteriores": "string",
                           "tratamientos_actuales": "string",
                           "tratamientos_futuros_previstos": "string",
                           "detalles_adicionales": "string",
                           "medicamentos_y_razon": "string",
                           "nombre_especialista": "string",
                           "especialidad_medica": "string",
                           "numero_colegiado": "string",
                           "direccion": "string",
                           "ciudad": "string",
                           "provincia": "string",
                           "telefono": "string",
                           "correo_electronico": "string",
                           "fecha": "string",
                           "firma": "string",
                            "sugerencia_tratamientos": [tratamiento: "string", explicacion: "string"],
                            "sugerencia_farmacos": [farmaco: "string", efecto: "string"],
                            "sugerencia_revision": [frecuencia_revision: "string", explicacion: "string"],
                       }.""")

    try:
        base64_image = encode_image(image)
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {my_api_key}"
        }

        payload = {
            "model": model,
            "messages": [
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text", "text": user_prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "temperature": temperature,
            "max_tokens": max_tokens,
            "top_p": top_p,
            "frequency_penalty": frequency_penalty,
            "presence_penalty": presence_penalty
        }

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        response_data = response.json()

        if 'choices' in response_data and len(response_data['choices']) > 0:
            return response_data['choices'][0]['message']['content']
        else:
            return "Error: No valid response from GPT-4."
    except Exception as e:
        return f"Error during GPT-4 image processing: {e}"

# Function to process and combine images and pass to GPT
def process_and_combine_images(pdf_file_path, orientation, model, temperature, max_tokens, top_p, frequency_penalty, presence_penalty):
    images = convert_pdf_to_images(pdf_file_path)
    if isinstance(images, str) and images.startswith("Error"):
        return images, []

    combined_image = combine_images(images, orientation)

    # Send combined image to GPT-4
    marked_fields = send_image_to_gpt(combined_image, model, temperature, max_tokens, top_p, frequency_penalty, presence_penalty)
    if "Error" in marked_fields:
        return marked_fields, []

    return marked_fields, [combined_image]

# Function to display images
def show_images(pdf_file_path):
    images = convert_pdf_to_images(pdf_file_path)
    if isinstance(images, str) and images.startswith("Error"):
        return []
    return images

# Function to save extracted fields to a JSON file
def save_to_json(extracted_fields):
    try:
        data = json.loads(extracted_fields)
        json_file = "extracted_data.json"

        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        return json_file
    except Exception as e:
        return f"Error during JSON export: {e}"

# Function to save extracted fields to an EXCEL file
def save_to_excel(extracted_fields, pdf_file_name):
    try:
        data = json.loads(extracted_fields)
        df = pd.DataFrame([data])
        df.insert(0, 'NOMBRE_ARCHIVO', pdf_file_name)
        excel_file = "extracted_data.xlsx"
        df.to_excel(excel_file, index=False)
        return excel_file
    except Exception as e:
        return f"Error during Excel export: {e}"

# Create the Gradio interface
def interface_fn(pdf_file_path, orientation, model, temperature, max_tokens, top_p, frequency_penalty, presence_penalty):
    text_output, images = process_and_combine_images(pdf_file_path, orientation, model, temperature, max_tokens, top_p, frequency_penalty, presence_penalty)
    return text_output, images

def show_images_fn(pdf_file_path):
    images = show_images(pdf_file_path)
    return gr.update(visible=True, value=images)

def download_json_fn(extracted_fields):
    json_file_path = save_to_json(extracted_fields)
    if json_file_path.startswith("Error"):
        return gr.update(visible=True, label=json_file_path)
    return json_file_path

def download_excel_fn(extracted_fields, pdf_file_path):
    pdf_file_name = os.path.basename(pdf_file_path)
    excel_file_path = save_to_excel(extracted_fields, pdf_file_name)
    if excel_file_path.startswith("Error"):
        return gr.update(visible=True, label=excel_file_path)
    return excel_file_path

def clear_fn():
    return None, gr.update(visible=False), gr.update(visible=False)


prompt_template_2 = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

model_2 = OpenAI(openai_api_key=openai.api_key, model_name="gpt-3.5-turbo-instruct")

prompt_2 = PromptTemplate(
    template=prompt_template_2, input_variables=["context", "question"]
)

chain = load_qa_chain(model_2, chain_type="stuff", prompt=prompt_2)



embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

Pregunta = ""

db = FAISS.load_local(folder_path="/teamspace/studios/this_studio/Curso_IA_Gen/Formularios",embeddings=embeddings,index_name="myFaissIndex",allow_dangerous_deserialization=True)
docs = db.similarity_search(Pregunta)

response = chain(
    {"input_documents": docs, "question": Pregunta}, return_only_outputs=True
)

def responder_cie(Pregunta):
    embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
    db = FAISS.load_local(folder_path="/teamspace/studios/this_studio/Curso_IA_Gen/Formularios",embeddings=embeddings,index_name="myFaissIndex",allow_dangerous_deserialization=True)
    docs = db.similarity_search(Pregunta)
    response = chain({"input_documents": docs, "question": Pregunta})
        
    return response['output_text']



/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [2]:

css = """
#fixed-pdf-input {
    width: 100%;  /* Set the desired width */
    height: 300px;  /* Set the desired height */
    overflow: hidden;  /* Hide any overflowing content */
    border: 1px solid #ccc;  /* Optional: Add a border */
    padding: 10px;  /* Optional: Add padding */
    box-sizing: border-box;  /* Ensure padding and border are included in the width and height */
}
"""


In [3]:
with gr.Blocks(css=css, theme=gr.themes.Default(primary_hue=gr.themes.colors.indigo).set(button_primary_background_fill="*primary_400")) as interface:
    with gr.Tab("Procesador PDF"):
        gr.Markdown(
            """
            # Extractor de Datos desde Archivos PDF
            Cargue un PDF para extraer e identificar las casillas marcadas mediante OCR y GPT-4. 
            Haga clic en «Enviar» para extraer los campos y en «Mostrar Imágenes» para mostrar las páginas PDF.
            """
        )
        with gr.Row():
            with gr.Column():
                pdf_input = gr.File(type="filepath", label="Cargar PDF", elem_id="fixed-pdf-input")
                orientation = gr.Dropdown(["vertical", "horizontal"], label="Orientación")
                submit_button = gr.Button("Enviar", variant="primary")
                clear_button = gr.Button("Limpiar")
            with gr.Column():
                text_output = gr.Textbox(label="Campos Extraídos", placeholder="La información extraída aparecerá aquí...", lines=23)
        
        image_gallery = gr.Gallery(label="Páginas PDF", visible=True)
        show_images_button = gr.Button("Mostrar Imágenes", visible=True)
        
        gr.Markdown(
            """
            # Asistente virtual
            Compruebe los códigos CIE-10 y devuelve la última versión publicada por la OMS
            """
        )
        with gr.Row():
            inputs = gr.Textbox(label="Pregunta", lines=5)
            outputs = gr.Textbox(label="Respuesta", lines=5)
        with gr.Row():
            enviar_button = gr.Button("Enviar", variant="primary")
            clear_button2 = gr.Button("Limpiar")
        
        enviar_button.click(fn=responder_cie, inputs=[inputs], outputs=[outputs])
        clear_button2.click(fn=lambda: (None, gr.update(visible=True), gr.update(visible=True)), outputs=[outputs])
        
        with gr.Row():
            download_json_button = gr.Button("Descargar en JSON", visible=True)
            download_excel_button = gr.Button("Descargar en Excel", variant="primary")

        # State variables for configuration
        model_state = gr.State(default_config["model"])
        temperature_state = gr.State(default_config["temperature"])
        max_tokens_state = gr.State(default_config["max_tokens"])
        top_p_state = gr.State(default_config["top_p"])
        frequency_penalty_state = gr.State(default_config["frequency_penalty"])
        presence_penalty_state = gr.State(default_config["presence_penalty"])

        submit_button.click(
            fn=interface_fn, 
            inputs=[pdf_input, orientation, model_state, temperature_state, max_tokens_state, top_p_state, frequency_penalty_state, presence_penalty_state],
            outputs=[text_output, image_gallery]
        )
        show_images_button.click(fn=show_images_fn, inputs=pdf_input, outputs=image_gallery)
        clear_button.click(fn=lambda: (None, gr.update(visible=True), gr.update(visible=True)), outputs=[text_output, show_images_button, image_gallery])
        download_json_button.click(fn=download_json_fn, inputs=[text_output], outputs=[gr.File()])
        download_excel_button.click(fn=download_excel_fn, inputs=[text_output, pdf_input], outputs=[gr.File()])

    with gr.Tab("Configuration"):
        gr.Markdown(
            """
            # Configuration Settings
            Adjust the GPT-4 parameters to fine-tune the extraction process.
            """
        )
        model = gr.Textbox(value=default_config["model"], label="Model")
        temperature = gr.Slider(0.0, 1.0, value=default_config["temperature"], step=0.1, label="Temperature")
        max_tokens = gr.Slider(10, 4000, value=default_config["max_tokens"], step=50, label="Max Tokens")
        top_p = gr.Slider(0.0, 1.0, value=default_config["top_p"], step=0.1, label="Top P")
        frequency_penalty = gr.Slider(-2.0, 2.0, value=default_config["frequency_penalty"], step=0.1, label="Frequency Penalty")
        presence_penalty = gr.Slider(-2.0, 2.0, value=default_config["presence_penalty"], step=0.1, label="Presence Penalty")

        save_button = gr.Button("Save Settings")
        
        save_button.click(
            fn=lambda model, temperature, max_tokens, top_p, frequency_penalty, presence_penalty: (model, temperature, max_tokens, top_p, frequency_penalty, presence_penalty),
            inputs=[model, temperature, max_tokens, top_p, frequency_penalty, presence_penalty],
            outputs=[model_state, temperature_state, max_tokens_state, top_p_state, frequency_penalty_state, presence_penalty_state]
        )

interface.launch(share=True)


Running on local URL:  http://127.0.0.1:7860


Running on public URL: https://5d3bcd851e54501861.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Matplotlib created a temporary cache directory at /tmp/matplotlib-fbswhhr7 because the default path (/teamspace/studios/this_studio/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
